# Twitter BOT

## per pubblicare i dati covid-19

https://www.youtube.com/watch?v=hgvn7aFGDHw

https://docs.tweepy.org/en/v3.5.0/getting_started.html

In [ ]:
# !pip install tweepy
# !pip install python-dotenv
# !pip install pandas

import tweepy
import os
import dotenv
from dotenv import load dotenv
load dotenv()

# parametri token salvati dentro .env {con permission write}

# AUTENTICAZIONE
auth = tweepy.OAuthHandler(os.environ["CONSUMER_KEY"], os.environ["CONSUMER_SECRET"])
auth.set_access_token(os.environ["ACCESS_TOKEN"], os.environ["ACCESS_TOKEN_SECRET"])
api = tweepy.API(auth)

# api.update_status("test") # x vedere se funziona posto "test"

# PRIMI 10 TWEET CHE CONTENGONO UN TESTO CERCATO
for tweet in tweepy.Cursor(api.search, q="MachineLearning").items(10):
    print(tweet.text) # caratteri da convertire in UTF8

# tutti i tweet pubblici in home page twitter per quell'account
for tweet in api.home_timeline():
    print(tweet.text)
    
import pandas as pd
import requests

df = pd.read_json("https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-json/dpc-covid19-ita-province.json")
print(len(df))
print(df)
# trasformo la colonna data in formato datetime
df['data'] = pd.to_datetime(df['data'])
print(df.dtypes)
# elenco province italiane
set(df['denominazione_provincia'])
# solo i dati della provincia di Ancona
ancona=df[df['denominazione_provincia']=='Ancona']
print(ancona[-1:]) # solo l'ultimo dato/riga più recente
print(ancona['totale_casi'].diff()) # riassunto totale dei casi nuovi - ordinati dai primi agli ultimi gg
print(ancona['totale_casi'].diff()[-30:]) # riassunto totale dei casi degli ultimi 30 gg

# somme cumulate di 7 record attuale+precedenti (min 1)
moving7daycases = ancona['totale_casi'].diff().rolling(min_periods=1,window=7).sum()

# GRAFICO andamento dei casi a settimana in prov. Ancona - si vedono 3 ondate e l'ultima impressionante
fig = moving7daycases.plot()
print(fig)
# nelle ascisse i numeri da 0 a 50000 sono gli indici del record (ma in realtà i record sono solo 365)

brescia=df[df['denominazione_provincia']=='Brescia']
nuovicasiBrescia = brescia['totale_casi'].diff() / 12 # rapporto ogni 100.000 abitanti, dato che Brescia ha 1.200.000 abitanti
fig2=nuovicasiBrescia.plot()
fig3=nuovicasiBrescia.plot.box() 
# MEDIA MOBILE vedo distribuzione intorno ai valori min max e media casi (ordinata: casi nuovi al gg)

# PROPHET modello previsionale
# ha bisogno di pystan - che a sua volta richiede un compilatore
# va fatto (in Windows 10) con particolare versione precedente e successivo upgrade
# !pip install pystan==2.17.1.0
# !pip install fbprophet==0.6
# !pip install --upgrade fbprophet
'''
from fbprophet import Prophet
m = Prophet()
m.fit(df)
future = m.make_future_dataframe(periods=365)
'''

# ipywidgets
# widget interattivi in HTML per jupyter notebooks e IPython kernel
# ad es. selection widget come dropdown per scegliere Città Provincia
'''
widgets.Dropdown(
    options=['1', '2', '3'],
    value='2', # default
    description='Number:', # label
    disabled=False,
)
'''

# RISULTATO FINALE: caricare una immagine in un Tweet Twitter
'''
tweet_text=input('il contenuto del post ')
image_path=input('il percorso del file immagine')
# generazione
status = api.update_with_media(image_path, tweet_text)
api.update_status(status-tweet_text)
'''